# Example notebook for performing data discovery using D3L

The D3L package is based on the ICDE 2020 paper with the same name: https://arxiv.org/pdf/2011.10427.pdf

In [1]:
import numpy as np
from d3l.indexing.similarity_indexes import NameIndex, FormatIndex, ValueIndex, EmbeddingIndex, DistributionIndex
from d3l.input_output.dataloaders import PostgresDataLoader, CSVDataLoader
from d3l.querying.query_engine import QueryEngine
from d3l.utils.functions import pickle_python_object, unpickle_python_object
from d3l.indexing.feature_extraction.values.glove_embedding_transformer import GloveTransformer
from d3l.indexing.feature_extraction.values.fasttext_embedding_transformer import FasttextTransformer

# 接下来继续使用GloveTransformer类进行处理


In [1]:
import numpy as np
import pandas as pd
import csv
from d3l.indexing.similarity_indexes import NameIndex, FormatIndex, ValueIndex, EmbeddingIndex, DistributionIndex
from d3l.input_output.dataloaders import PostgresDataLoader, CSVDataLoader
from d3l.querying.query_engine import QueryEngine
from d3l.utils.functions import pickle_python_object, unpickle_python_object
from d3l.indexing.feature_extraction.values.glove_embedding_transformer import GloveTransformer
from d3l.indexing.feature_extraction.values.fasttext_embedding_transformer import FasttextTransformer

## Data indexing

There are two main steps in D3L: indexing and searching. 
Data indexing reads all datasets from a given data lake and indexes their columns using multiple LSH indexes and multiple types of similarity evidence.

### Data loading

This example cell uses a CSV data loader that requires a root path where all CSV files to index can be found. In addition Pandas-specific CSV reading arguments can be passed, such as the separator character, encoding, etc.

Other supported data loader types include Postgresql data loaders.

In [4]:
# Postgres data loader

# dataloader = PostgresDataLoader(
#     db_host="localhost",
#     db_password="postgres",
#     db_username="postgres",
#     db_name="postgres",
#     db_port=5432
# )

In [2]:
# CSV data loader
dataloader = CSVDataLoader(
    root_path='G:/small/csv_benchmark',
    sep=','
)

In [2]:
# CSV data loader
dataloader = CSVDataLoader(
    root_path='G:/large/data_ssd/webtable/large/split_1/split_1',
    sep=','
)

In [2]:
# CSV data loader
dataloader = CSVDataLoader(
    root_path='/home/wangyanzhang/split1_query', #这个要改
    sep=','
)
print("LOADED!")


LOADED!


In [2]:
# CSV data loader
dataloader = CSVDataLoader(
    root_path='/home/wangyanzhang/d3l-main/small/csv_benchmark', #这个要改
    sep=','
)
print("LOADED!")



LOADED!


### Index creation
D3L uses five types of column similarity evidence and, therefore, five LSH indexes: name-based, format-based, value-based, embedding-based, and distribution-based. The last index type is characteristic to numerical columns and performs LSH indexing using the random projections hash function applied on the density-based histogram of values. This is different from the numerical approach proposed in the paper.

Note that the embedding index requires to downloading and loading into memory the pretrained GloVe embedding vectors. These will be downloaded once in the working directory or int the configured cache directory. The default zipped embedding vector file has 1.7 GB and its unzipped version has 4.7 GB.

In [3]:
# Create new indexes
name_index = NameIndex(dataloader=dataloader)
pickle_python_object(name_index, './name.lsh')
print("Name: SAVED!")

format_index = FormatIndex(dataloader=dataloader)
pickle_python_object(format_index, './format.lsh')
print("Format: SAVED!")

value_index = ValueIndex(dataloader=dataloader)
pickle_python_object(value_index, './value.lsh')
print("Value: SAVED!")





100%|██████████| 1530/1530 [00:27<00:00, 54.93it/s] 


Name: SAVED!


100%|██████████| 1530/1530 [50:02<00:00,  1.96s/it]  


Format: SAVED!


100%|██████████| 1530/1530 [00:33<00:00, 45.79it/s] 


Value: SAVED!


In [5]:
embedding_index = EmbeddingIndex(dataloader=dataloader)


File exists. Use --overwrite to download anyway.


100%|██████████| 1530/1530 [00:52<00:00, 29.39it/s] 


In [8]:
embedding_index = EmbeddingIndex(dataloader=dataloader,index_cache_dir="./")     ###下载的

File exists. Use --overwrite to download anyway.


100%|██████████| 1530/1530 [00:52<00:00, 28.98it/s]


In [9]:
pickle_python_object(embedding_index, './embedding.lsh')
print("Embedding: SAVED!")

Embedding: SAVED!


In [7]:

distribution_index = DistributionIndex(dataloader=dataloader)
pickle_python_object(distribution_index, './distribution.lsh')
print("Distribution: SAVED!")

100%|██████████| 1530/1530 [00:27<00:00, 55.73it/s] 


Distribution: SAVED!


In [3]:
# Create new indexes
name_index = NameIndex(dataloader=dataloader)
pickle_python_object(name_index, './name_1.lsh')
print("Name: SAVED!")

format_index = FormatIndex(dataloader=dataloader)
pickle_python_object(format_index, './format_1.lsh')
print("Format: SAVED!")

value_index = ValueIndex(dataloader=dataloader)
pickle_python_object(value_index, './value_1.lsh')
print("Value: SAVED!")
embedding_index = EmbeddingIndex(dataloader=dataloader)
pickle_python_object(embedding_index, './embedding_1.lsh')
print("Embedding: SAVED!")

distribution_index = DistributionIndex(dataloader=dataloader)
pickle_python_object(distribution_index, './distribution_1.lsh')
print("Distribution: SAVED!")





  7%|▋         | 210762/2959101 [2:03:47<26:54:20, 28.37it/s]  


ParserError: Error tokenizing data. C error: out of memory

In [3]:
# Or load them from disk if they have been created
name_index = unpickle_python_object('./name_benchmark_mul.lsh')
format_index = unpickle_python_object('./format_benchmark_mul.lsh')
value_index = unpickle_python_object('./value_benchmark_mul.lsh')
embedding_index = unpickle_python_object('./embedding_benchmark_mul.lsh')
distribution_index = unpickle_python_object('./distribution_benchmark_mul.lsh')

In [3]:
#  load them from disk if they have been created
name_index = unpickle_python_object('./name_webtable_large_split1&query.lsh')
print("Name Unpickled!")



Name Unpickled!


In [4]:
format_index = unpickle_python_object('./format_webtable_large_split1&query.lsh')
print("Format Unpickled!")

Format Unpickled!


In [5]:
value_index = unpickle_python_object('./value_webtable_large_split1&query.lsh')              #这些要改
print("Value Unpickled!")

Value Unpickled!


In [6]:
embedding_index = unpickle_python_object('./embedding_webtable_large_split1&query.lsh')
print("Embedding Unpickled!")

Embedding Unpickled!


In [7]:
distribution_index = unpickle_python_object('./distribution_webtable_large_split1&query.lsh')
print("Distribution Unpickled!")
print("Index LOADED!")


Distribution Unpickled!
Index LOADED!


## Data searching

Given a query table, each of the columns will be added to the relevant indexes and a top-k nearest neighbor search is performed for each query column on each index. The results are then aggregated as discussed in the paper, except for the aggregation of the five similarity scores which, in this example, is performed by a simple mean.

If no aggregation is passed, each result will consist of a (result table name, five similarity scores) tuple. Otherwise, each result will consist of a (result table name, aggregated similarity score value) tuple. In the first case, the order of the score types corresponds to the order of the indexes passed to the query engine object.


In [11]:
import pandas as pd
# 读取CSV文件
data = pd.read_csv('G:/small/query_table.csv')
# 获取第一列的所有值，假设第一列名为'table_name'(query_table)
table_names = data['query_table'].tolist()


In [6]:
print(table_names)

['t_1934eacab8c57857____c10_0____0', 't_1934eacab8c57857____c10_0____1', 't_1934eacab8c57857____c10_0____2', 't_1934eacab8c57857____c10_0____3', 't_1934eacab8c57857____c10_0____4', 't_1934eacab8c57857____c10_1____0', 't_1934eacab8c57857____c10_1____1', 't_1934eacab8c57857____c10_1____2', 't_1934eacab8c57857____c10_1____3', 't_1934eacab8c57857____c10_1____4', 't_1934eacab8c57857____c11_0____0', 't_1934eacab8c57857____c11_0____1', 't_1934eacab8c57857____c11_0____2', 't_1934eacab8c57857____c11_0____3', 't_1934eacab8c57857____c11_0____4', 't_1934eacab8c57857____c11_1____0', 't_1934eacab8c57857____c11_1____1', 't_1934eacab8c57857____c11_1____2', 't_1934eacab8c57857____c11_1____3', 't_1934eacab8c57857____c11_1____4', 't_1934eacab8c57857____c12_0____0', 't_1934eacab8c57857____c12_0____1', 't_1934eacab8c57857____c12_0____2', 't_1934eacab8c57857____c12_0____3', 't_1934eacab8c57857____c12_0____4', 't_1934eacab8c57857____c12_1____0', 't_1934eacab8c57857____c12_1____1', 't_1934eacab8c57857____c12_

In [58]:
results_list = []
qe = QueryEngine(name_index, format_index, value_index, embedding_index, distribution_index)
for table_name in table_names:
    results, extended_results = qe.table_query(
        table=dataloader.read_table(table_name=table_name),
        aggregator=None, k=10, verbose=True
    )
    results_list.append((results, extended_results))


In [7]:
import csv

In [65]:
output_file = 'results.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Result', 'Extended Result'])
    for results, extended_results in results_list:
        for result, extended_result in zip(results, extended_results):
            writer.writerow([table_name, result, extended_result])

print(f"Results written to {output_file}.")

ValueError: not enough values to unpack (expected 3, got 2)

In [63]:
for results, extended_results in results_list:
    print(f"Results:")
    print(results)
    print("Extended Results:")
    print(extended_results)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
print("Start Querying!")
qe = QueryEngine(name_index, format_index, value_index, embedding_index, distribution_index)
print("qe start")
table_name='t_1f6b65ebc2ab3ed3____c2_0____3'  #这个要改（查询表表名）
print(table_name)
results, extended_results = qe.table_query(table=dataloader.read_table(table_name='t_1f6b65ebc2ab3ed3____c2_0____3'),
                                           aggregator=None, k=10, verbose=True)
print("Get results!")

Start Querying!
qe start
t_1f6b65ebc2ab3ed3____c2_0____3
Get results!


In [5]:
for result in extended_results:
    query_table = table_name
    candidate_table = result[0]

    for x, column_info in enumerate(result[1]):
        column_info_name = f"column_info{x+1}"
        globals()[column_info_name] = column_info[0]

        #print(column_info[0])  # 输出 column_info[x][0]
        #print(column_info[1])  # 输出 column_info[x][1]
        print(f"{query_table}, {candidate_table}.csv, {column_info[0][0]}, {column_info[0][1]}")




file_path = '/home/wangyanzhang/d3l-main/d3l-main/examples/notebooks/results_mul.csv'  #过渡用的查询结果表

with open(file_path, mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)


     # Write the header
    writer.writerow(['query_table', 'candidate_table', 'query_col_name', 'candidate_col_name'])

    for result in extended_results:
        query_table = table_name
        candidate_table = result[0]

        for x, column_info in enumerate(result[1]):
            column_info_name = f"column_info{x+1}"
            globals()[column_info_name] = column_info[0]

            row = [f"{query_table}.csv", f"{candidate_table}.csv", column_info[0][0], column_info[0][1]]
            writer.writerow(row)
print("CSV file has been created successfully.")




t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____3.csv, Day, Day
, Month5ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____3.csv, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c1_0____1.csv, Day, Day
, Dayb65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c1_0____1.csv, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c2_0____0.csv, Day, Day
, Month5ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c2_0____0.csv, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____1.csv, Day, Day
, Month5ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____1.csv, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_0____4.csv, Day, Day
, Dayb65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_0____4.csv, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_0____0.csv, Day, Day
, Dayb65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_0____0.csv, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c4_0____4.csv, 

In [ ]:
# 读取包含真实候选表的CSV文件
groundtruth_file = "/data_ssd/webtable/small_query/ground_truth.csv"
groundtruth_df = pd.read_csv(groundtruth_file, usecols=["query_table", "candidate_table"])

# 读取查询到的候选表的CSV文件
results_file = "/home/wangyanzhang/d3l-main/d3l-main/examples/notebooks/results.csv"
results_df = pd.read_csv(results_file, usecols=["candidate_table"])

#print(table_name)
# 现有的查询表名
table_name = table_name+".csv"
print(table_name)
# 在真实候选表中找到对应的候选表集合Tq，并去重
Tq = set(groundtruth_df[groundtruth_df["query_table"] == table_name]["candidate_table"].unique())

# 在查询到的候选表中找到对应的候选表集合Tq'，并去重
Tq_prime = set(results_df["candidate_table"].unique())
print("真实候选表集合Tq:")
print(Tq)
print("查询到的候选表集合Tq':")
print(Tq_prime)
# 计算准确率和召回率
precision = len(Tq.intersection(Tq_prime)) / len(Tq_prime)
print("准确率:", precision)
recall = len(Tq.intersection(Tq_prime)) / len(Tq)

#print("准确率:", precision)
print("召回率:", recall)

In [87]:
qe = QueryEngine(name_index, format_index, value_index, embedding_index, distribution_index)
table_name='t_1f6b65ebc2ab3ed3____c2_0____3'
results, extended_results = qe.table_query(table=dataloader.read_table(table_name='t_1f6b65ebc2ab3ed3____c2_0____3'),
                                           aggregator=None, k=50, verbose=True)

In [88]:
print("Results:")
print(results)
print("Extended Results:")
print(extended_results)
print("分隔")

print(f"{table_name}, {extended_results[1][0]}, {', '.join(map(str, extended_results[1][1][1][0]))}")
print("分割")
print('query_table,','candidate_table,','query_col_name,','candidate_col_name')
for result in extended_results:
    query_table=table_name
    candidate_table = result[0]
    column_info1 = result[1][0][0]
    #column_info2 = result[1][1][0]
    print(column_info1)
    #print(column_info2)
    print(f"{query_table}.csv, {candidate_table}.csv,{column_info1[0]}, {column_info1[1]}")



#Results返回的是表名，代码并没有实现聚合以计算表相似度，所以Results返回的是空
#而Extended Results返回的是与之对应的列的相似度，故有返回值

Results:
[('t_1f6b65ebc2ab3ed3____c4_0____3', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c4_0____2', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c3_1____0', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c3_1____2', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c3_1____1', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c2_0____0', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c4_0____1', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c2_0____3', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c2_0____1', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c3_1____4', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c4_0____0', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c4_0____4', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c2_0____2', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c3_1____3', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c2_0____4', [1.0, 1.0, 0.0, 0.0, 0.0]), ('t_1f6b65ebc2ab3ed3____c2_1__

##这是将查询表，候选表，对应的查询列，对应的候选列依次输出，如果目标表与湖中表有多个相关列，则依次输出

In [89]:
for result in extended_results:
    query_table = table_name
    candidate_table = result[0]

    for x, column_info in enumerate(result[1]):
        column_info_name = f"column_info{x+1}"
        globals()[column_info_name] = column_info[0]

        #print(column_info[0])  # 输出 column_info[x][0]
        #print(column_info[1])  # 输出 column_info[x][1]
        print(f"{query_table}, {candidate_table}.csv, {column_info[0][0]}, {column_info[0][1]}")


t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c4_0____3.csv, Day, Day
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c4_0____3.csv, Month, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c4_0____2.csv, Day, Day
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c4_0____2.csv, Month, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____0.csv, Day, Day
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____0.csv, Month, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____2.csv, Day, Day
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____2.csv, Month, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____1.csv, Day, Day
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c3_1____1.csv, Month, Month
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c2_0____0.csv, Day, Day
t_1f6b65ebc2ab3ed3____c2_0____3, t_1f6b65ebc2ab3ed3____c2_0____0.csv, Month, Month
t_1f6b65ebc2ab3ed3____c2_0__

##这段弃用了，有问题

In [65]:
import csv

file_path = 'H:/桌面/98_k_50.csv'

# Open the CSV file in write mode
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(['query_table', 'candidate_table', 'query_col_name', 'candidate_col_name'])

    # Write the data rows
    for result in extended_results:
        query_table = table_name
        candidate_table = result[0]
        column_info = result[1][0][0]
        writer.writerow([query_table+".csv",candidate_table+".csv", column_info[0], column_info[1]])

print("CSV file has been created successfully.")


CSV file has been created successfully.


##这段也有问题

In [76]:
import csv

file_path = 'H:/桌面/98_k_50.csv'

# Open the CSV file in write mode
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(['query_table', 'candidate_table', 'query_col_name', 'candidate_col_name'])

    # Write the data rows
    for y, column_info in enumerate(result[1]):
        column_info_name = f"column_info{y+1}"
        globals()[column_info_name] = column_info[0]

        #print(column_info[0])  # 输出 column_info[x][0]
        #print(column_info[1])  # 输出 column_info[x][1]
        print(f"{query_table}, {candidate_table}.csv, {column_info[0][0]}, {column_info[0][1]}")
        writer.writerow([query_table,candidate_table+".csv", column_info[0][0], column_info[0][1]])

print("CSV file has been created successfully.")


t_1f6b65ebc2ab3ed3____c2_0____3.csv, t_1934eacab8c57857____c12_0____3.csv, Day, Continent name
t_1f6b65ebc2ab3ed3____c2_0____3.csv, t_1934eacab8c57857____c12_0____3.csv, Month, Continent name
CSV file has been created successfully.


##这段是正确的，对于有多个匹配列的表会重复输出候选表名，后续统计Tq‘时会去重

In [91]:
import csv

file_path = 'H:/桌面/98_k_50.csv'

with open(file_path, mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)


     # Write the header
    writer.writerow(['query_table', 'candidate_table', 'query_col_name', 'candidate_col_name'])

    for result in extended_results:
        query_table = table_name
        candidate_table = result[0]

        for x, column_info in enumerate(result[1]):
            column_info_name = f"column_info{x+1}"
            globals()[column_info_name] = column_info[0]

            row = [f"{query_table}.csv", f"{candidate_table}.csv", column_info[0][0], column_info[0][1]]
            writer.writerow(row)
print("CSV file has been created successfully.")


CSV file has been created successfully.


In [9]:
# 定义查询表的真实候选表集合Tq
# 用你实际的数据替换以下示例集合。
Tq = {'t_1f6b65ebc2ab3ed3____c1_0____3', 't_1f6b65ebc2ab3ed3____c4_0____1', 't_1f6b65ebc2ab3ed3____c3_1____4'}

# 定义算法预测的候选表集合Tq'
# 用你实际的数据替换以下示例集合。
Tq_prime = {'t_1f6b65ebc2ab3ed3____c4_0____1', 't_1f6b65ebc2ab3ed3____c3_1____4', 't_1f6b65ebc2ab3ed3____c1_0____0'}

# 计算准确率和召回率
precision = len(Tq.intersection(Tq_prime)) / len(Tq_prime)
recall = len(Tq.intersection(Tq_prime)) / len(Tq)

print("准确率:", precision)
print("召回率:", recall)


准确率: 0.6666666666666666
召回率: 0.6666666666666666


In [92]:
import pandas as pd

# 读取包含真实候选表的CSV文件
groundtruth_file = "G:/small/csv_groundtruth/alignment_groundtruth.csv"
groundtruth_df = pd.read_csv(groundtruth_file, usecols=["query_table", "candidate_table"])

# 读取查询到的候选表的CSV文件
results_file = "H:/桌面/98_k_50.csv"
results_df = pd.read_csv(results_file, usecols=["candidate_table"])

#print(table_name)
# 现有的查询表名
table_name = table_name+".csv"
print(table_name)
# 在真实候选表中找到对应的候选表集合Tq，并去重
Tq = set(groundtruth_df[groundtruth_df["query_table"] == table_name]["candidate_table"].unique())

# 在查询到的候选表中找到对应的候选表集合Tq'，并去重
Tq_prime = set(results_df["candidate_table"].unique())
print("真实候选表集合Tq:")
print(Tq)
print("查询到的候选表集合Tq':")
print(Tq_prime)
# 计算准确率和召回率
precision = len(Tq.intersection(Tq_prime)) / len(Tq_prime)
print("准确率:", precision)
recall = len(Tq.intersection(Tq_prime)) / len(Tq)

#print("准确率:", precision)
print("召回率:", recall)


t_1f6b65ebc2ab3ed3____c2_0____3
t_1f6b65ebc2ab3ed3____c2_0____3.csv
真实候选表集合Tq:
{'t_1f6b65ebc2ab3ed3____c3_1____0.csv', 't_1f6b65ebc2ab3ed3____c2_1____0.csv', 't_1f6b65ebc2ab3ed3____c4_0____4.csv', 't_1f6b65ebc2ab3ed3____c4_0____0.csv', 't_1f6b65ebc2ab3ed3____c3_0____2.csv', 't_1f6b65ebc2ab3ed3____c4_0____3.csv', 't_1f6b65ebc2ab3ed3____c3_1____4.csv', 't_1f6b65ebc2ab3ed3____c4_0____1.csv', 't_1f6b65ebc2ab3ed3____c2_0____2.csv', 't_1f6b65ebc2ab3ed3____c2_0____0.csv', 't_1f6b65ebc2ab3ed3____c2_0____3.csv', 't_1f6b65ebc2ab3ed3____c1_0____0.csv', 't_1f6b65ebc2ab3ed3____c3_0____4.csv', 't_1f6b65ebc2ab3ed3____c2_1____1.csv', 't_1f6b65ebc2ab3ed3____c3_0____1.csv', 't_1f6b65ebc2ab3ed3____c2_1____2.csv', 't_1f6b65ebc2ab3ed3____c1_0____1.csv', 't_1f6b65ebc2ab3ed3____c2_0____4.csv', 't_1f6b65ebc2ab3ed3____c2_1____4.csv', 't_1f6b65ebc2ab3ed3____c3_1____3.csv', 't_1f6b65ebc2ab3ed3____c2_1____3.csv', 't_1f6b65ebc2ab3ed3____c3_0____3.csv', 't_1f6b65ebc2ab3ed3____c1_0____2.csv', 't_1f6b65ebc2ab3ed3____

In [61]:
# 请注意，这里假设results是一个包含元组的列表，每个元组包含(query_table_name, candidate_table_name, query_column, candidate_column, aggregated_score)的信息

print("query_table\tcandidate_table\tquery_col_name\tcandidate_col_name")
for query_table_name, candidate_table_name, query_column, candidate_column, aggregated_score in extended_results:
    print(f"{query_table_name}\t{candidate_table_name}\t{query_column}\t{candidate_column}")


query_table	candidate_table	query_col_name	candidate_col_name


ValueError: not enough values to unpack (expected 5, got 2)

## Extracting table similarity scores for alternative usage

Given a query table, one can extract the similarity scores that describe the relatedness relationship between the query and its neighbours and use these scores in other tasks. For example, these scores can be aggregated using a function of choice and the result can be construed as a distance between the tables, i.e., used in clustering.

In the cell above `results` will be a collection of pairs, with each item containing a related (to the query table) table name and five similarity scores, one for each type of backend. All five scores are returned because no `aggregation` was used.

The `verbose` parameter leads to the return of `extended_results` that, for each related table, offers a deeper view into column-level similarities. The format of each item of `extended_results` is (<*table_name*>, [((<*query_column_name*>, <*related_column_name*>), [*column_level_scores*])]). The order of the scores is given by the order of the backends passed to the `QueryEngine` object.

## Querying individual indexes

Occasionally, it may be useful to interrogate individual indexes. Before exemplifying the process for doing so, it is important to mention that each index stores in its buckets similar **columns**. In other words, every operation performed on individual indexes will operate at **column-level**.

Every index inherits a *SimilarityIndex* base class that exposes a *query* method. The signature of the method and its docstring can be consulted in the *similarity_indexes* sub-package of *indexing*.

In [17]:
# Name index query
results = name_index.query(query="month", k=10) # The query arg should be a column name. Tokenization will be performed automatically.
print(results)

[('t_1f6b65ebc2ab3ed3____c2_0____1.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c3_1____4.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c2_1____2.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c2_0____0.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c4_0____3.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c2_1____1.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c2_1____3.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c4_0____2.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c3_1____3.Month', 1.0), ('t_1f6b65ebc2ab3ed3____c2_0____2.Month', 1.0)]


In [43]:
# Format index query
results = format_index.query(query="<title,id,price>", k=10) # The query arg should be a collection of string values. The corresponding format descriptors will be extracted automatically.
print(results)

[]


In [20]:
# Value index query
results = value_index.query(query="<0,1,2,3,4>", k=10) # The query arg should be a collection of string values. Value pre-processing will be performed automatically.
print(results)

[]


In [17]:
# Embeddings index query
results = embedding_index.query(query="<list/set>", k=10) # The query arg should be a collection of string values. The corresponding embeddings will be extracted automatically.
print(results)

[]


In [18]:
# Distribution index query
results = distribution_index.query(query="<list/set>", k=10) # The query arg should be a collection of numerical values. The corresponding distribution will be extracted automatically.
print(results)

[]


In each case of the above *k* is optional. If not passed all identified neighbours of the query will be returned.
Lastly, each item in *results* will be a pair of the form *<(table_name.column_name, similarity_score)>*.